# Drowsiness Detection Feature Extraction

In [ ]:
# Import Modules
from imutils import face_utils
import dlib
import cv2
import math
import numpy as np


## Helper Functions

In [ ]:
# Helper Functions

# returns angle between two vectors
def angle_of_vectors(a,b):
     dotProduct = a[0]*b[0] + a[1]*b[1]
     modOfVector1 = math.sqrt( a[0]*a[0] + a[1]*a[1])*math.sqrt(b[0]*b[0] + b[1]*b[1]) 
     angle = dotProduct/modOfVector1
     if abs(angle) > 1.0:
         return 0 # avoid domain errors
     return math.degrees(math.acos(angle))

# returns tuple of centroid coordinates
def calculate_centroid(points):
    sumx = 0
    sumy = 0
    for (x,y) in points:
        sumx += x
        sumy += y  
    return (sumx/len(points), sumy/len(points))

# code for evaluating pitch angle of the head pose taken from
# https://learnopencv.com/head-pose-estimation-using-opencv-and-dlib/,
# https://github.com/opencv/opencv/blob/3.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/Utils.cpp#L189,
# and https://docs.opencv.org/3.1.0/dc/d2c/tutorial_real_time_pose.html

def rot2euler(rotation):
    euler = [0, 0, 0] # roll, pitch, yaw
    if rotation[1][0] > 0.998:
        euler[0] = 0
        euler[1] = np.pi / 2
        euler[2] = np.arctan2(rotation[0][2], rotation[2][2])
    elif rotation[1][0] < -0.998:
        euler[0] = 0
        euler[1] = -np.pi / 2
        euler[2] = np.arctan2(rotation[0][2], rotation[2][2])
    else:
        euler[0] = np.arctan2(-rotation[1][2], rotation[1][1])
        euler[1] = np.arcsin(rotation[1][0])
        euler[2] = np.arctan2(-rotation[2][0], rotation[0][0])
    return euler

def calculate_pitch(points, size):
    #3D model points
    model_points = np.array([
        (0.0, 0.0, 0.0),             # Nose tip
        (0.0, -330.0, -65.0),        # Chin
        (-225.0, 170.0, -135.0),     # Right eye outer corner
        (225.0, 170.0, -135.0),      # Left eye outer corner
        (-150.0, -150.0, -125.0),    # Right mouth outer corner
        (150.0, -150.0, -125.0)      # Left mouth outer corner
    ])
    
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]], dtype = "double"
    )
    
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

    angles = rot2euler(cv2.Rodrigues(rotation_vector)[0])
    return angles[1]

## Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Landmark Declarations

p = "/content/drive/My Drive/Drowsiness Detection/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)
(L_start, L_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(R_start, R_end) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(M_start, M_end) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

Mounted at /content/drive


## Perform Feature Extraction on Dataset

In [ ]:
from google.colab.patches import cv2_imshow

# Main
person = 4
mode = 0
number = 0
sequence = []
count = 0
cap = cv2.VideoCapture('/content/drive/My Drive/Drowsiness Detection/Dataset/0' + str(person) + '/' + str(mode) + '/' + str(number) + '.mov')

while True:
    good, image = cap.read()
    if not good:
        break
    if count % 100 == 0:
        print(count, 'frames processed for', person, mode, number)
    count += 1

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    
    # no face detected
    if len(rects) == 0:
        temp = np.empty((8,))
        temp[:] = np.nan
        sequence.append(temp)
        continue

    for (i, rect) in enumerate(rects):
        feature_vector = [] # left eye angle, right eye angle, mouth angle, left pupil, right pupil, head pose pitch

        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        lefteye = shape[L_start : L_end]
        righteye = shape[R_start : R_end]
        mouth = shape[M_start : M_end]
        centroids = [calculate_centroid(lefteye), calculate_centroid(righteye)]
        pupils = []

        # Eye Angle (inner corner)
        LA = lefteye[1] - lefteye[0]
        LB = lefteye[5] - lefteye[0]
        RA = righteye[2] - righteye[3]
        RB = righteye[4] - righteye[3]

        feature_vector.append(angle_of_vectors(LA,LB))
        feature_vector.append(angle_of_vectors(RA,RB))
        
        
        # Mouth Angle (average of inner mouth angles)
        inner_mouth = mouth[12:]
        LMA = inner_mouth[3] - inner_mouth[4]
        LMB = inner_mouth[5] - inner_mouth[4]
        LM = angle_of_vectors(LMA,LMB)
        RMA = inner_mouth[1] - inner_mouth[0]
        RMB = inner_mouth[7] - inner_mouth[0]
        RM = angle_of_vectors(RMA,RMB) 

        feature_vector.append((LM + RM) / 2.0)

        # Pupil 
        for eye in [lefteye, righteye]:
            xs = [p[0] for p in eye]
            ys = [p[1] for p in eye] 
            start = (min(xs), min(ys))
            end = (max(xs), max(ys))

            gray_eye = gray[start[1]:end[1],start[0]:end[0]]
            rows, cols = gray_eye.shape
            gray_eye = cv2.GaussianBlur(gray_eye, (11, 11), 0)
            gray_eye = cv2.medianBlur(gray_eye, 3)

            threshold = cv2.threshold(gray_eye, 127, 255, cv2.THRESH_BINARY_INV)[1]
            contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

            for cnt in contours:
                (x, y, w, h) = cv2.boundingRect(cnt)
                px = x + w/2 + start[0]
                py = y + h/2 + start[1]
                pupils.append((px,py))
                cv2.circle(image, (int(px), int(py)), 2, (255, 0, 0), 2)
                break
        # left pupil
        feature_vector.append(pupils[0][0] - centroids[0][0])
        feature_vector.append(pupils[0][1] - centroids[0][1])
        # right pupil
        if len(pupils) < 2:
            feature_vector.append(np.nan)
            feature_vector.append(np.nan)
        else:
            feature_vector.append(pupils[1][0] - centroids[1][0])
            feature_vector.append(pupils[1][1] - centroids[1][1])

        # Head Pitch Angle
        image_points = np.array([
            tuple(shape[33]),          # Nose tip
            tuple(shape[8]),           # Chin
            tuple(righteye[0]),        # Right eye outer corner
            tuple(lefteye[3]),         # Left eye outer corner
            tuple(mouth[0]),           # Right mouth outer corner
            tuple(mouth[6])            # Left mouth outer corner
        ], dtype="double")

        feature_vector.append(calculate_pitch(image_points, gray.shape))

        sequence.append(feature_vector)
        # print(feature_vector)
        break

    # cv2_imshow(image)

# save extracted features to csv
sequence = np.array(sequence)
print(sequence.shape)
np.savetxt(str(person) + '_' + str(mode) + '_' + str(number) + ".csv", sequence, delimiter=",")

cv2.destroyAllWindows()
cap.release()

0 frames processed for 4 0 0
100 frames processed for 4 0 0
200 frames processed for 4 0 0
300 frames processed for 4 0 0
400 frames processed for 4 0 0
500 frames processed for 4 0 0
600 frames processed for 4 0 0
700 frames processed for 4 0 0
800 frames processed for 4 0 0
900 frames processed for 4 0 0
1000 frames processed for 4 0 0
1100 frames processed for 4 0 0
1200 frames processed for 4 0 0
1300 frames processed for 4 0 0
1400 frames processed for 4 0 0
1500 frames processed for 4 0 0
(1501, 8)
